In [19]:
import pandas as pd
import sklearn

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## 1 cargamos datos y hacemos el split

In [2]:
dataset = pd.read_csv('./datasets/whr2017.csv')
print(dataset.describe())

             rank       score        high         low         gdp      family  \
count  155.000000  155.000000  155.000000  155.000000  155.000000  155.000000   
mean    78.000000    5.354019    5.452326    5.255713    0.984718    1.188898   
std     44.888751    1.131230    1.118542    1.145030    0.420793    0.287263   
min      1.000000    2.693000    2.864884    2.521116    0.000000    0.000000   
25%     39.500000    4.505500    4.608172    4.374955    0.663371    1.042635   
50%     78.000000    5.279000    5.370032    5.193152    1.064578    1.253918   
75%    116.500000    6.101500    6.194600    6.006527    1.318027    1.414316   
max    155.000000    7.537000    7.622030    7.479556    1.870766    1.610574   

           lifexp     freedom  generosity  corruption    dystopia  
count  155.000000  155.000000  155.000000  155.000000  155.000000  
mean     0.551341    0.408786    0.246883    0.123120    1.850238  
std      0.237073    0.149997    0.134780    0.101661    0.500028 

In [6]:
# creamos variables x y y
X = dataset[['gdp', 'family', 'lifexp', 'freedom',
        'corruption', 'generosity', 'dystopia']]
y = dataset[['score']]

print(X.shape, y.shape)

(155, 7) (155, 1)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25)

## 2 entrenamos los modelos de regresion lineal
LinearRegression, Lasso y Ridge son algoritmos de regresion lineal, en el fondo son una formula matematica, a diferencia de LinearRegression,  Lasso y Ridge para evitar el overfitting, se aumenta el sesgo al aplicar una ponderacion a las features mas importantes.   
por tal motivo entrenaremos cada modelo con el mismo set de datos para ver como se desempeña cada uno. 

In [56]:
# Aquí definimos nuestros regresores uno por 1 y llamamos el fit o ajuste 
modelLinear = LinearRegression().fit(X_train, y_train)
y_predict_linear = modelLinear.predict(X_test)

# Alpha, que es valor labda y entre mas valor tenga alpha mas penalizacion a la variable dara 
# Lasso
modelLasso = Lasso(alpha=0.02).fit(X_train, y_train)
y_predict_lasso = modelLasso.predict(X_test)

# Ridge
modelRidge = Ridge(alpha=1).fit(X_train, y_train)
y_predict_ridge = modelRidge.predict(X_test)

## 3 evaluacion del modelo 
usaremos el error cuadratico medio (MSE) para evaluar que tan bien predice cada modelo.

In [58]:
linear_loss = mean_squared_error(y_test, y_predict_linear)
#Loss lineal 
print("Linear loss: %.10f" % float(linear_loss))
# Loss Lasso
lasso_loss = mean_squared_error(y_test, y_predict_lasso)
print("Lasso Loss: %.10f" % float(lasso_loss))
# Loss Ridge
ridge_loss = mean_squared_error(y_test, y_predict_ridge)
print("Ridge loss: %.10f" % float(ridge_loss))

Linear loss: 0.0000001028
Lasso Loss: 0.0723252212
Ridge loss: 0.0070235725


podemos ver que el modelo con mejores resultados para estos datos es LinearRegression()

## coeficientes de ponderacion 
algo importante que nos regresan estos modelos son los coeficientes de ponderacion, los cuales son vectores con un valor correspondiente a  cada columna(feature) de nuestro dataset.
entre mas alto sea el valor del coeficiente mas importancia esa feature aporto al modelo. 


In [59]:
print('Coef LASSO')
print(modelLasso.coef_)
print('='*32)
print('Coef RIDGE')
print(modelRidge.coef_)

Coef LASSO
[1.41909837 0.90129388 0.31267336 0.76453761 0.         0.06364152
 0.88535439]
Coef RIDGE
[[1.10343263 0.96477217 0.83058961 0.88410941 0.65268328 0.7262513
  0.95797724]]


vemos como en LASSO hay coeficientes = 0 
para simplicficar la comparacion por columnas crearemos un dataframe con los datos resultantes

In [60]:
coef = pd.DataFrame({"modelRidge": list(modelRidge.coef_[0]),
"modelLasso": list(modelLasso.coef_)},
index=['gdp', 'family', 'lifexp', 'freedom',
        'corruption', 'generosity', 'dystopia'])
coef

,modelRidge,modelLasso
gdp,1.103433,1.419098
family,0.964772,0.901294
lifexp,0.830590,0.312673
freedom,0.884109,0.764538
corruption,0.652683,0.000000
generosity,0.726251,0.063642
dystopia,0.957977,0.885354


## concluciones 
- el indice de ingresos tiene mayor peso que las otras variables, para calcular el indice de la felicidad
- sorprendentemente vemos como el indice de corrupcion tiene poco impacto en en el inidce de la felicidad segun Ridge y Lasso